# Sækja ákveðna pakka sem eru nauðsynlegir

In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


# Forvinnsla gagna

In [3]:
from datasets import Dataset
import json

def load_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def prepare_dataset(data):
    texts = [
        f"Recipe: {recipe['name']}\nIngredients:\n" +
        "\n".join(recipe['ingredients']) +
        "\n\n" for recipe in data
    ]
    return Dataset.from_dict({"text": texts})

recipes = load_data('recipes.json')
dataset = prepare_dataset(recipes)

# Þjálfun GPT-2 módels

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

def tokenize_function(examples):
    tokens = tokenizer(examples['text'], padding='max_length', truncation=True)
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens


# Tokenize the data set
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    #output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    warmup_steps=10,
    prediction_loss_only=True
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset #This is our dataset
)

# Fine-tune the model
trainer.train()

Map:   0%|          | 0/1022 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: freyrthegod26 (freyrthegod26-university-of-iceland) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.846100
20,0.388000
30,0.310800
40,0.257100
50,0.262300
60,0.245800
70,0.283200
80,0.266400
90,0.227600
100,0.260600


TrainOutput(global_step=511, training_loss=0.33025739369677004, metrics={'train_runtime': 365.3025, 'train_samples_per_second': 2.798, 'train_steps_per_second': 1.399, 'total_flos': 534080913408000.0, 'train_loss': 0.33025739369677004, 'epoch': 1.0})

# Búa til nýja uppskrift

In [56]:
def generate_recipe(prompt, required_ingredients, max_length=300):
    # Tokenize and encode the prompt with attention mask
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt', padding=True, truncation=True).to('cuda')

    # Generate the recipe with sampling enabled
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.8,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Split into lines
    lines = recipe.split('\n')

    # Initialize sections
    ingredients, instructions = [], []
    current_section = None

    # Parse the results
    for line in lines:
        if "ingredients" in line.lower():
            current_section = 'ingredients'
            if not ingredients:
                ingredients.append("Ingredients:")
        elif "instructions" in line.lower():
            current_section = 'instructions'
            if not instructions:
                instructions.append("Instructions:")
        elif current_section == 'ingredients':
            if line.strip():
                ingredients.append(line)
        elif current_section == 'instructions' and line.strip():
            instructions.append(line)

    # Ensure required ingredients are explicitly mentioned with quantities
    default_quantities = {ingredient.lower(): "200g or 1 cup" for ingredient in required_ingredients}
    for ingredient in required_ingredients:
        if not any(ingredient.lower() in line.lower() for line in ingredients):
            quantity = default_quantities.get(ingredient.lower(), "as desired")
            ingredients.append(f"{ingredient.capitalize()} ({quantity})")

    # Generate additional ingredients if needed
    desired_ingredient_count = len(required_ingredients) + 2
    additional_ingredients_needed = max(0, min(6, desired_ingredient_count) - len(ingredients) + 1)

    if additional_ingredients_needed > 0:
        more_outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=50,
            temperature=0.8,
            do_sample=True,
            top_k=50,
            top_p=0.95
        )
        additional_text = tokenizer.decode(more_outputs[0], skip_special_tokens=True)
        additional_lines = additional_text.split('\n')
        for line in additional_lines:
            if line.strip() and "ingredients" not in line.lower() and "instructions" not in line.lower():
                ingredients.append(line)
                if len(ingredients) >= 3 + additional_ingredients_needed:
                    break

    # Combine the sections
    formatted_recipe = "\n".join(ingredients + instructions)
    return formatted_recipe

# Gather input from the user
new_recipe_prompt = "Create a creative recipe using your selected ingredients."
user_ingredients = input("Enter the ingredients you want to use, separated by commas: ").split(',')
user_ingredients = [ingredient.strip() for ingredient in user_ingredients]

print(generate_recipe(new_recipe_prompt, user_ingredients))

Enter the ingredients you want to use, separated by commas: beef,rice
Ingredients:
1 tbsp vegetable oil, plus extra for frying
2 onions (about 1/3 red), sliced into thin wedges
Beef (200g or 1 cup)
Rice (200g or 1 cup)
